In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import requests
from time import sleep

import sys
from collections import defaultdict # 콜렉션에서 불러옵니다.


In [11]:
#chromdriver='/Users/seonghunjeong/Downloads/chromedriver'
chromdriver=None

In [31]:
if not chromdriver:
    driver = webdriver.Chrome()
else:
    driver = webdriver.Chrome('/Users/seonghunjeong/Downloads/chromedriver')

In [32]:
def parser(driver, target_address):
        
    num = target_address.split('/')[6]
    
    driver.get(target_address)
    
    try:
        title = driver.find_elements_by_xpath("//div[@id='Main']/div[2]/div/div/div/h1")[0].text
        d[num]['title'] = title

        desc = driver.find_elements_by_xpath("//div[@id='Main']/div[2]/div/div/div[3]/p")[0].text
        d[num]['description'] = desc

        d[num]['site'] = target_address

        ad = driver.find_element_by_css_selector('.HeadingList-col:nth-child(2)')
        if ad.text.startswith('Additional Information'):
            d[num]['PHASE'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(2) .HeadingList-item:nth-child(1) > .HeadingList-value').text
            d[num]['STUDY SPONSOR'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(2) .HeadingList-item:nth-child(2) > .HeadingList-value').text
            d[num]['STUDY DRUGS'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(2) .HeadingList-item:nth-child(3) > .HeadingList-value').text
        else:
            d[num]['PHASE'] = None
            d[num]['STUDY SPONSOR'] = None
            d[num]['STUDY DRUGS'] = None

    
        d[num]['STUDY TYPE'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(1) .HeadingList-item:nth-child(1) > .HeadingList-value').text
        d[num]['RANDOMIZED STUDY'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(1) .HeadingList-item:nth-child(2) > .HeadingList-value').text
        d[num]['PLACEBO CONTROLLED'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(1) .HeadingList-item:nth-child(3) > .HeadingList-value').text
        d[num]['LENGTH OF PARTICIPATION'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(1) .HeadingList-item:nth-child(4) > .HeadingList-value').text
        d[num]['NUMBER OF STUDY VISITS'] = driver.find_element_by_css_selector('.HeadingList-col:nth-child(1) .HeadingList-item:nth-child(5) > .HeadingList-value').text

        d[num]['AGE'] = driver.find_element_by_css_selector('.u-hide-narrow .HeadingList-item:nth-child(1) > .HeadingList-value').text
        d[num]['MUTATION(S)'] = driver.find_element_by_css_selector('.u-hide-narrow .HeadingList-item:nth-child(2) > .HeadingList-value').text
        d[num]['FEV1% PREDICTED'] = driver.find_element_by_css_selector('.u-hide-narrow .HeadingList-item:nth-child(3) > .HeadingList-value').text
    
        Other_Primary_Eligibility_Criteria = driver.find_elements_by_xpath("//div[@id='Main']/div[2]/div/div/div[6]/p[2]")
        #if Other_Primary_Eligibility_Criteria:
        #    d[num]['Other Primary Eligibility Criteria'] = driver.find_element_by_css_selector('.u-border-top:nth-child(8) > p:nth-child(3)').text
        #else:
        #    d[num]['Other Primary Eligibility Criteria'] = None
        
        sr =  driver.find_elements_by_css_selector('.HeadingList-item:nth-child(1) p:nth-child(2)')
        if sr:
            d[num]['WHAT WE LEARNED'] = sr[0].text
        else:
            d[num]['WHAT WE LEARNED'] = None


        EFFECTIVENESS = driver.find_elements_by_css_selector('.u-border-top:nth-child(6) .HeadingList-item:nth-child(2) > .HeadingList-value:nth-child(2)')
        if EFFECTIVENESS:

            d[num]['EFFECTIVENESS'] = EFFECTIVENESS[0].text.replace('EFFECTIVENESS:\n','')
        else:
            d[num]['EFFECTIVENESS'] = None
    except OSError as err:
        print("OS error: {0}".format(err))
    except ValueError:
        print("Could not convert data to an integer.")
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise

    driver.implicitly_wait(1)

In [33]:
target_address = 'https://www.cff.org/Trials/Finder/details/519/Phase-3-study-of-VX-445-triple-combination-drug-in-people-with-CF-ages-12-years-and-older-who-have-two-copies-of-the-F508del-mutation'
parser(driver, target_address)

In [34]:
%%time

d = defaultdict(dict)

f = open('target_list.txt', 'r')
lines = f.readlines()
for line in lines:
#    target_address = 'https://www.cff.org/Trials/Finder/details/278/Home-Monitoring-of-Lung-Function'

    target_address = line.strip()
    
    parser(driver, target_address)
    

f.close()

#driver.quit()

CPU times: user 3.98 s, sys: 326 ms, total: 4.31 s
Wall time: 8min 44s


In [35]:
import json
with open('collection.json', 'w') as f:
    json.dump(d, f, ensure_ascii=False)

In [37]:
df = pd.read_json('collection.json')

In [42]:
len(df.T)

160

In [43]:
writer = pd.ExcelWriter('collection.xlsx')
df.T.to_excel(writer,'Sheet1')
writer.save()